In [ ]:
import pickle
import shapely
from glob import glob
import os
from os.path import join, exists
import pandas as pd
import xarray as xr
import rioxarray as rxa

# Add main repo to path
import sys
from os.path import expanduser
sys.path.append(expanduser('../../'))

from spicy_snow.retrieval import retrieve_snow_depth

from spicy_snow.IO.user_dates import get_input_dates

In [ ]:
get_input_dates('2023-03-10')

In [ ]:
area = shapely.geometry.box(-150, 68, -149, 69)